In [8]:
!ls

colleges.db	       stanford_sql_exercises.Rmd
movies_and_ratings.db  stanford_sql_exercises.ipynb


In [9]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('movies_and_ratings.db')

In [10]:
pd.read_sql_query("select * from sqlite_master where type = 'table'", conn)

,type,name,tbl_name,rootpage,sql
0,table,Movie,Movie,2,"CREATE TABLE Movie(mID int, title text, year i..."
1,table,Reviewer,Reviewer,3,"CREATE TABLE Reviewer(rID int, name text)"
2,table,Rating,Rating,4,"CREATE TABLE Rating(rID int, mID int, stars in..."
